In [69]:
import pandas as pd

In [75]:
df = pd.read_parquet('tiling_results.parquet')

In [76]:
df.shape

(350, 12)

In [77]:
len(df[df.tiling_compatible == True]) 

86

In [78]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0  cant_extract_variables    110
1      unsupported_format    100
2                    None     86
3  tile_generation_failed     42
4                 timeout     10
5        no_granule_found      2


In [79]:
df[(df.tiling_compatible == False) & (df.incompatible_reason.isna())]

,collection_concept_id,concept_id,data_url,backend,format,extension,tiling_compatible,incompatible_reason,error_message,tiles_url,variable,data_variables
